In [32]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score
from scipy.stats import ks_2samp
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
%matplotlib inline

In [2]:
X_train_res = pd.read_csv('database/X_train_res.csv').drop(columns=['Unnamed: 0'])
y_train_res = pd.read_csv('database/y_train_res.csv').drop(columns=['Unnamed: 0'])
X_val = pd.read_csv('database/X_val.csv').drop(columns=['INDEX'])
y_val = pd.read_csv('database/y_val.csv', names=['INDEX', 'IND_BOM_1_1']).drop(columns=['INDEX'])
X_test = pd.read_csv('database/X_test.csv').drop(columns=['INDEX'])
y_test = pd.read_csv('database/y_test.csv', names=['INDEX', 'IND_BOM_1_1']).drop(columns=['INDEX'])

In [3]:
input_dim = len(list(X_train_res))

<h2>Treinando MLPs</h2>
<p>Para o treino de MLPs foram testadas diversas formas de otimizar hiperparâmetros como: learning rate, algoritmo de otimização, número de camadas escondidas, etc. A estimativa do learning rate e do número de camadas escondidas foi feita automaticamente a partir do conjunto de validação.</p>

<h3>O MLP Básico</h3>
<p>O MLP é montado com base na biblioteca Keras e segue uma estrutura básica:</p>
<ul>
    <li>O MLP em si é uma classe Sequential e nela serão adicionadas as camadas de entrada, saída e internas.</li>
    <li>As camadas são uma classe Dense. Passamos para o construtor dessa classe o número de nodes, a função de ativação e, no caso de ser a primeira camada, a dimensão da entrada.</li>
    <li>Antes de treinar o MLP, devemos compilar o mesmo e setar parâmetros como a função de otimização, algumas métricas opcionais como 'accuracy' e a função de perda, que retorna uma métrica para avaliarmos o MLP</li>
    <li>Para treinarmos o modelo utilizaremos a função fit que recebe como parâmetros X sendo as amostras para treinamento, y sendo as classes de cada amostra, batch_size sendo o comprimento do batch utilizado no treinamento, o número de epochs sendo quantas vezes iremos dar entrada com o dataset para o treinamento, validation_data sendo os dados utilizados para validação e callbacks para pararmos o treinamento com certas condições.</li>
</ul>
<p>Para a primeira fase, procuraremos funções de ativação, número de nós na camada de entrada e função de otimização que melhorem o acerto no conjunto de validação</p>

<h3>Estimando Hiperparâmetros</h3>
<ul>
    <li>Função de Ativação</li>
    <li>Função de Otimização</li>
    <li>Camadas escondidas</li>
</ul>

<h4>Função de Ativação nas Camadas de Entrada</h4>

In [23]:
mlp = Sequential()
mlp.add(Dense(20, activation='tanh', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6387 - acc: 0.6349 - val_loss: 0.6529 - val_acc: 0.6093
Epoch 2/100000
307646/307646 [==============================] - 9s 30us/step - loss: 0.6227 - acc: 0.6523 - val_loss: 0.6222 - val_acc: 0.6409
Epoch 3/100000
307646/307646 [==============================] - 9s 30us/step - loss: 0.6134 - acc: 0.6619 - val_loss: 0.6635 - val_acc: 0.6054
Epoch 4/100000
307646/307646 [==============================] - 9s 29us/step - loss: 0.6033 - acc: 0.6715 - val_loss: 0.6181 - val_acc: 0.6520
Epoch 5/100000
307646/307646 [==============================] - 9s 29us/step - loss: 0.5942 - acc: 0.6780 - val_loss: 0.6286 - val_acc: 0.6452
Epoch 6/100000
307646/307646 [==============================] - 9s 30us/step - loss: 0.5862 - acc: 0.6830 - val_loss: 0.6384 - val_acc: 0.6315
Epoch 7/100000
307646/307646 [==============================] - 9s 29us/step - loss: 0.580

In [25]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Acurácia Média de Validação: 0.64


In [26]:
mlp = Sequential()
mlp.add(Dense(30, activation='tanh', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6378 - acc: 0.6367 - val_loss: 0.6471 - val_acc: 0.6133
Epoch 2/100000
307646/307646 [==============================] - 9s 30us/step - loss: 0.6207 - acc: 0.6546 - val_loss: 0.6279 - val_acc: 0.6369
Epoch 3/100000
307646/307646 [==============================] - 9s 30us/step - loss: 0.6096 - acc: 0.6645 - val_loss: 0.6541 - val_acc: 0.6150
Epoch 4/100000
307646/307646 [==============================] - 9s 31us/step - loss: 0.5953 - acc: 0.6774 - val_loss: 0.6332 - val_acc: 0.6355
Epoch 5/100000
307646/307646 [==============================] - 9s 30us/step - loss: 0.5823 - acc: 0.6854 - val_loss: 0.6117 - val_acc: 0.6581
Epoch 6/100000
307646/307646 [==============================] - 9s 31us/step - loss: 0.5729 - acc: 0.6909 - val_loss: 0.6223 - val_acc: 0.6508
Epoch 7/100000
307646/307646 [==============================] - 9s 30us/step - loss: 0.566

In [27]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Acurácia Média de Validação: 0.63


In [28]:
mlp = Sequential()
mlp.add(Dense(50, activation='tanh', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 10s 34us/step - loss: 0.6377 - acc: 0.6370 - val_loss: 0.6409 - val_acc: 0.6227
Epoch 2/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6211 - acc: 0.6537 - val_loss: 0.6057 - val_acc: 0.6603
Epoch 3/100000
307646/307646 [==============================] - 11s 34us/step - loss: 0.6104 - acc: 0.6652 - val_loss: 0.6192 - val_acc: 0.6449
Epoch 4/100000
307646/307646 [==============================] - 10s 34us/step - loss: 0.5976 - acc: 0.6770 - val_loss: 0.6186 - val_acc: 0.6513
Epoch 5/100000
307646/307646 [==============================] - 10s 33us/step - loss: 0.5832 - acc: 0.6875 - val_loss: 0.6590 - val_acc: 0.6079


In [29]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Acurácia Média de Validação: 0.63


In [30]:
mlp = Sequential()
mlp.add(Dense(100, activation='tanh', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 13s 41us/step - loss: 0.6361 - acc: 0.6367 - val_loss: 0.6351 - val_acc: 0.6256
Epoch 2/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.6185 - acc: 0.6568 - val_loss: 0.6325 - val_acc: 0.6301
Epoch 3/100000
307646/307646 [==============================] - 12s 38us/step - loss: 0.6070 - acc: 0.6685 - val_loss: 0.6712 - val_acc: 0.5963
Epoch 4/100000
307646/307646 [==============================] - 12s 38us/step - loss: 0.5938 - acc: 0.6805 - val_loss: 0.6317 - val_acc: 0.6400
Epoch 5/100000
307646/307646 [==============================] - 12s 38us/step - loss: 0.5776 - acc: 0.6920 - val_loss: 0.6151 - val_acc: 0.6576
Epoch 6/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.5630 - acc: 0.7003 - val_loss: 0.6172 - val_acc: 0.6556
Epoch 7/100000
307646/307646 [==============================] - 12s 38us/step - loss:

In [31]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Acurácia Média de Validação: 0.64


<p>Pela configuração atual notamos que o número de nós na camada de entrada não influenciou muito na perda sobre o conjunto de validação, porém 30 obteve um valor mais baixo de 0.63. Manteremos esse valor.</p>

In [36]:
mlp = Sequential()
mlp.add(Dense(100, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 14s 44us/step - loss: 0.6413 - acc: 0.6330 - val_loss: 0.6264 - val_acc: 0.6404
Epoch 2/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.6276 - acc: 0.6466 - val_loss: 0.6303 - val_acc: 0.6302
Epoch 3/100000
307646/307646 [==============================] - 14s 45us/step - loss: 0.6180 - acc: 0.6571 - val_loss: 0.6813 - val_acc: 0.5801
Epoch 4/100000
307646/307646 [==============================] - 14s 46us/step - loss: 0.6083 - acc: 0.6668 - val_loss: 0.6276 - val_acc: 0.6396


In [37]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Acurácia Média de Validação: 0.64


In [38]:
mlp = Sequential()
mlp.add(Dense(100, activation='relu', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 14s 46us/step - loss: 0.6314 - acc: 0.6433 - val_loss: 0.6429 - val_acc: 0.6159
Epoch 2/100000
307646/307646 [==============================] - 12s 40us/step - loss: 0.6119 - acc: 0.6648 - val_loss: 0.6307 - val_acc: 0.6332
Epoch 3/100000
307646/307646 [==============================] - 12s 38us/step - loss: 0.5967 - acc: 0.6793 - val_loss: 0.6360 - val_acc: 0.6288
Epoch 4/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.5821 - acc: 0.6900 - val_loss: 0.6298 - val_acc: 0.6393
Epoch 5/100000
307646/307646 [==============================] - 14s 44us/step - loss: 0.5690 - acc: 0.6982 - val_loss: 0.6194 - val_acc: 0.6569
Epoch 6/100000
307646/307646 [==============================] - 12s 38us/step - loss: 0.5587 - acc: 0.7043 - val_loss: 0.6146 - val_acc: 0.6647
Epoch 7/100000
307646/307646 [==============================] - 12s 40us/step - loss:

In [40]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Acurácia Média de Validação: 0.63


<p>Modificando as funções de ativação na camada de entrada também não variou muito a perda média. Testamos então na camada de saída.</p>

In [46]:
mlp = Sequential()
mlp.add(Dense(100, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='relu')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 14s 45us/step - loss: 6.3356 - acc: 0.5250 - val_loss: 10.5645 - val_acc: 0.3446
Epoch 2/100000
307646/307646 [==============================] - 13s 42us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 10.5645 - val_acc: 0.3446
Epoch 3/100000
307646/307646 [==============================] - 13s 42us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 10.5645 - val_acc: 0.3446
Epoch 4/100000
307646/307646 [==============================] - 13s 42us/step - loss: 8.0590 - acc: 0.5000 - val_loss: 10.5645 - val_acc: 0.3446


In [47]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Acurácia Média de Validação: 10.56


In [44]:
mlp = Sequential()
mlp.add(Dense(100, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='tanh')) # Camada de saída
mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 12s 41us/step - loss: 0.6493 - acc: 0.6261 - val_loss: 0.6383 - val_acc: 0.6350
Epoch 2/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.6314 - acc: 0.6451 - val_loss: 0.6723 - val_acc: 0.5946
Epoch 3/100000
307646/307646 [==============================] - 12s 40us/step - loss: 0.6187 - acc: 0.6563 - val_loss: 0.6526 - val_acc: 0.6146
Epoch 4/100000
307646/307646 [==============================] - 14s 45us/step - loss: 0.6076 - acc: 0.6672 - val_loss: 0.6352 - val_acc: 0.6330
Epoch 5/100000
307646/307646 [==============================] - 13s 43us/step - loss: 0.5951 - acc: 0.6769 - val_loss: 0.6264 - val_acc: 0.6447
Epoch 6/100000
307646/307646 [==============================] - 12s 40us/step - loss: 0.5806 - acc: 0.6873 - val_loss: 0.6421 - val_acc: 0.6383
Epoch 7/100000
307646/307646 [==============================] - 13s 41us/step - loss:

In [45]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Acurácia Média de Validação: 0.64


<p>Notamos que utilizando ReLU como função de ativação na camada de saída não gera bons resultados, mas na camada de entrada obteve a menor perda. Por fim optamos por manter ReLU na camada de entrada e Sigmoid na de saída.</p>

<h4>Algoritmo de Otimização</h4>

In [48]:
mlp = Sequential()
mlp.add(Dense(100, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='tanh')) # Camada de saída
mlp.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 14s 45us/step - loss: 0.6602 - acc: 0.6067 - val_loss: 0.6258 - val_acc: 0.6511
Epoch 2/100000
307646/307646 [==============================] - 13s 43us/step - loss: 0.6475 - acc: 0.6272 - val_loss: 0.7155 - val_acc: 0.5736
Epoch 3/100000
307646/307646 [==============================] - 11s 37us/step - loss: 7.1530 - acc: 0.5144 - val_loss: 5.4931 - val_acc: 0.6554
Epoch 4/100000
307646/307646 [==============================] - 11s 37us/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.4931 - val_acc: 0.6554


In [49]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Acurácia Média de Validação: 3.08


In [50]:
mlp = Sequential()
mlp.add(Dense(100, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='tanh')) # Camada de saída
mlp.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 14s 45us/step - loss: 0.6479 - acc: 0.6272 - val_loss: 0.7031 - val_acc: 0.5805
Epoch 2/100000
307646/307646 [==============================] - 15s 47us/step - loss: 0.6430 - acc: 0.6335 - val_loss: 0.6784 - val_acc: 0.6076
Epoch 3/100000
307646/307646 [==============================] - 15s 47us/step - loss: 0.6389 - acc: 0.6382 - val_loss: 0.6796 - val_acc: 0.5912
Epoch 4/100000
307646/307646 [==============================] - 14s 45us/step - loss: 0.6309 - acc: 0.6456 - val_loss: 0.6575 - val_acc: 0.6104
Epoch 5/100000
307646/307646 [==============================] - 12s 40us/step - loss: 0.6270 - acc: 0.6498 - val_loss: 0.6443 - val_acc: 0.6222
Epoch 6/100000
307646/307646 [==============================] - 13s 42us/step - loss: 0.6217 - acc: 0.6541 - val_loss: 0.6298 - val_acc: 0.6391
Epoch 7/100000
307646/307646 [==============================] - 13s 43us/step - loss:

In [51]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Perda Média de Validação: 0.65


In [54]:
mlp = Sequential()
mlp.add(Dense(30, activation='sigmoid', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='tanh')) # Camada de saída
mlp.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 11s 37us/step - loss: 0.6451 - acc: 0.6288 - val_loss: 0.6404 - val_acc: 0.6250
Epoch 2/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6303 - acc: 0.6456 - val_loss: 0.6036 - val_acc: 0.6661
Epoch 3/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6219 - acc: 0.6531 - val_loss: 0.6252 - val_acc: 0.6357
Epoch 4/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6155 - acc: 0.6598 - val_loss: 0.6303 - val_acc: 0.6357
Epoch 5/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6078 - acc: 0.6678 - val_loss: 0.6395 - val_acc: 0.6296


In [55]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Perda Média de Validação: 0.63


<p>Notamos que a função adadelta gerou uma menor perda.</p>

In [58]:
mlp = Sequential()
mlp.add(Dense(30, activation='relu', input_dim=input_dim)) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 11s 35us/step - loss: 0.6358 - acc: 0.6384 - val_loss: 0.6713 - val_acc: 0.5815
Epoch 2/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6178 - acc: 0.6575 - val_loss: 0.6412 - val_acc: 0.6187
Epoch 3/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.6059 - acc: 0.6694 - val_loss: 0.6468 - val_acc: 0.6198
Epoch 4/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.5956 - acc: 0.6778 - val_loss: 0.6698 - val_acc: 0.6004
Epoch 5/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.5877 - acc: 0.6817 - val_loss: 0.6123 - val_acc: 0.6598
Epoch 6/100000
307646/307646 [==============================] - 10s 32us/step - loss: 0.5819 - acc: 0.6846 - val_loss: 0.6098 - val_acc: 0.6694
Epoch 7/100000
307646/307646 [==============================] - 10s 32us/step - loss:

In [59]:
print("Perda Média de Validação: %.2f"%(np.mean(history.history['val_loss'])))

Perda Média de Validação: 0.64


<p>No fim não houve muita mudança e os valores otimizados para cada teste gerou um valor próximo de todos os outros testes (0.64)</p>

<h4>Camadas Escondidas</h4>

In [5]:
hidden_nodes = 30

In [60]:
for hidden_layers in range(6):
    mlp = Sequential()
    mlp.add(Dense(30, activation='relu', input_dim=input_dim)) # Camada de entrada
    for i in range(hidden_layers):
        mlp.add(Dense(hidden_nodes, activation='relu')) # Camada de entrada
    mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
    mlp.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['acc'])
    history = mlp.fit(X_train_res, y_train_res, batch_size=64, epochs=100000, 
            callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
    print('Para %d camadas escondidas:\n\tPerda Média: %.2f'%(hidden_layers, np.mean(history.history['val_loss'])))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 13s 42us/step - loss: 0.6385 - acc: 0.6350 - val_loss: 0.6683 - val_acc: 0.5958
Epoch 2/100000
307646/307646 [==============================] - 11s 36us/step - loss: 0.6213 - acc: 0.6532 - val_loss: 0.6388 - val_acc: 0.6216
Epoch 3/100000
307646/307646 [==============================] - 11s 37us/step - loss: 0.6124 - acc: 0.6628 - val_loss: 0.6175 - val_acc: 0.6486
Epoch 4/100000
307646/307646 [==============================] - 12s 40us/step - loss: 0.6053 - acc: 0.6707 - val_loss: 0.6550 - val_acc: 0.6126
Epoch 5/100000
307646/307646 [==============================] - 11s 36us/step - loss: 0.5968 - acc: 0.6787 - val_loss: 0.6747 - val_acc: 0.5967
Epoch 6/100000
307646/307646 [==============================] - 12s 39us/step - loss: 0.5884 - acc: 0.6836 - val_loss: 0.6188 - val_acc: 0.6515
Para 0 camadas escondidas:
	Perda Média: 0.65
Train on 307646 samples, validate on 26

<p>O erro médio se manteve entre os mesmos valores [0.63, 0.65], porém notamos um acerto maior no conjunto de validação quando utilizamos 3 camadas escondidas.</p>

<h4>Batch Size</h4>

In [64]:
opt_hidden_layer = 3
mlp = Sequential()
mlp.add(Dense(30, activation='relu', input_dim=input_dim)) # Camada de entrada
for i in range(opt_hidden_layer):
    mlp.add(Dense(hidden_nodes, activation='relu')) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=128, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
print('Perda Média: %.2f'%(np.mean(history.history['val_loss'])))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 9s 29us/step - loss: 0.6366 - acc: 0.6351 - val_loss: 0.6661 - val_acc: 0.5969
Epoch 2/100000
307646/307646 [==============================] - 8s 26us/step - loss: 0.6177 - acc: 0.6572 - val_loss: 0.6440 - val_acc: 0.6269
Epoch 3/100000
307646/307646 [==============================] - 8s 26us/step - loss: 0.6076 - acc: 0.6674 - val_loss: 0.6452 - val_acc: 0.6223
Epoch 4/100000
307646/307646 [==============================] - 8s 26us/step - loss: 0.5995 - acc: 0.6756 - val_loss: 0.6459 - val_acc: 0.6319
Epoch 5/100000
307646/307646 [==============================] - 8s 26us/step - loss: 0.5907 - acc: 0.6833 - val_loss: 0.6634 - val_acc: 0.6143
Perda Média: 0.65


In [6]:
opt_hidden_layer = 3
mlp = Sequential()
mlp.add(Dense(30, activation='relu', input_dim=input_dim)) # Camada de entrada
for i in range(opt_hidden_layer):
    mlp.add(Dense(hidden_nodes, activation='relu')) # Camada de entrada
mlp.add(Dense(1, activation='sigmoid')) # Camada de saída
mlp.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['acc'])
history = mlp.fit(X_train_res, y_train_res, batch_size=32, epochs=100000, 
        callbacks=[EarlyStopping(patience=3)], validation_data=(X_val, y_val))
print('Perda Média: %.2f'%(np.mean(history.history['val_loss'])))

Train on 307646 samples, validate on 26077 samples
Epoch 1/100000
307646/307646 [==============================] - 17s 54us/step - loss: 0.6336 - acc: 0.6397 - val_loss: 0.6490 - val_acc: 0.6185
Epoch 2/100000
307646/307646 [==============================] - 13s 44us/step - loss: 0.6129 - acc: 0.6631 - val_loss: 0.6194 - val_acc: 0.6510
Epoch 3/100000
307646/307646 [==============================] - 13s 44us/step - loss: 0.5956 - acc: 0.6755 - val_loss: 0.6396 - val_acc: 0.6322
Epoch 4/100000
307646/307646 [==============================] - 14s 45us/step - loss: 0.5794 - acc: 0.6849 - val_loss: 0.6212 - val_acc: 0.6556
Epoch 5/100000
307646/307646 [==============================] - 14s 44us/step - loss: 0.5684 - acc: 0.6916 - val_loss: 0.6170 - val_acc: 0.6552
Epoch 6/100000
307646/307646 [==============================] - 14s 46us/step - loss: 0.5604 - acc: 0.6960 - val_loss: 0.6316 - val_acc: 0.6452
Epoch 7/100000
307646/307646 [==============================] - 14s 44us/step - loss:

<p>Para o tamanho do batch = 32 obtivemos a menor perda.</p>

In [ ]:
test_evaluation = mlp.evaluate(X_test, y_test, batch_size=32)
print("Conjunto de Teste:\n\tPerda: %.3f\n\tAcurácia: %.3f"%(test_evaluation[0],test_evaluation[1]))

In [82]:
y_pred = mlp.predict_classes(X_test)
y_pred_proba = mlp.predict(X_test)

In [105]:
# Método auxiliar para imprimir todas as métricas para um classificador
def generateMetrics(y_true, y_pred, y_pred_proba = None):
    print("Acurácia: %.5f"%(accuracy_score(y_true=y_true, y_pred=y_pred)))
    print("Recall: %.5f"%(recall_score(y_true=y_true, y_pred=y_pred)))
    print("Precisão: %.5f"%(precision_score(y_true=y_true, y_pred=y_pred)))
    print("F1-Score: %.5f"%(f1_score(y_true=y_true, y_pred=y_pred)))
    if not y_pred_proba is None:
        print("Área Sobre a Curva ROC: %.5f"%(roc_auc_score(y_true=y_true, y_score=y_pred_proba)))
        print("Precisão Média sobre as Probabilidades:%.5f"%(average_precision_score(y_true=y_true, y_score=y_pred_proba)))
        print("Matriz de Confusão:\n")
        print(pd.DataFrame(confusion_matrix(y_true=y_true, y_pred=y_pred), 
                           columns=['P', 'N'], 
                           index=['P', 'N']))

In [106]:
print('Métricas Geradas para o MLP:')
generateMetrics(y_test, y_pred, y_pred_proba)

Métricas Geradas para o MLP:
Acurácia: 0.65165
Recall: 0.71463
Precisão: 0.74385
F1-Score: 0.72895
Área Sobre a Curva ROC: 0.68076
Precisão Média sobre as Probabilidades:0.79633
Matriz de Confusão:

       P      N
P  23535  20717
N  24023  60160


In [114]:
def calc_distr(y_true:pd.DataFrame, y_pred_proba:np.array):
    ac_distr_0 = np.zeros(101)
    ac_distr_1 = np.zeros(101)
    count_classes = y_true['IND_BOM_1_1'].value_counts()
    for i in range(1, 101):
        lim = i/100.0
        count_classes = y_true[y_pred_proba <= lim]['IND_BOM_1_1'].value_counts()
        ac_distr_0[i] += count_classes[0]
        ac_distr_1[i] += count_classes[1]
    return (ac_distr_0/count_classes[0], ac_distr_1/count_classes[1])

In [116]:
distr_0, distr_1 = calc_distr(y_test, y_pred_proba)

In [117]:
ks_2samp(distr_0, distr_1)

Ks_2sampResult(statistic=0.17821782178217827, pvalue=0.07148265768062076)

In [120]:
roc_auc_score(y_true=y_test, y_score=y_pred_proba)

0.6813878353811542